In [1]:
import pandas as pd
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import json

BAM_MAPPINGS_FILE = '/home/groups/dpwall/briannac/general_data/bam_mappings.csv'


# Setup dataframe of nuclear families

In [4]:
bam_mappings = pd.read_csv(BAM_MAPPINGS_FILE, sep='\t', index_col=1)
bam_mappings = bam_mappings[bam_mappings['status']=='Passed_QC_analysis_ready']
sample_id_to_participant = {sample_id:participant_id for participant_id, sample_id in zip(bam_mappings.participant_id, bam_mappings.index)}
#bam_mappings = bam_mappings[['_' not in s for s in list(bam_mappings.index)]]

In [5]:
bam_mappings[bam_mappings['status']!='Passed_QC_analysis_ready']

,status,participant_id,family,AGRE Blood,NIMH Blood,relationship,FatherID,MotherID,sex_numeric,derived_affected_status,...,number_of_indels,number_of_homozygous_snvs,number_of_heterozygous_snvs,het_to_hom_ratio,ti_tv_ratio,dbsnp_overlap,concordance_with_genotyping_chip,cryptic_relatedness_check,family_based_average_coverage,familial_relationship_check
sample_id,,,,,,,,,,,,,,,,,,,,,


In [6]:
#family_idxs = bam_mappings[bam_mappings.relationship=='sibling'].groupby(['family', 'MotherID', 'FatherID']).aggregate(sum).index
fam_info_list = []
fams_phased = [f.split('/')[-1].replace('.phased.txt', '') for f in glob('/oak/stanford/groups/dpwall/users/kpaskov/PhasingFamilies/phased_ihart/*.phased.txt')]
family_idxs = bam_mappings[bam_mappings.relationship=='sibling'].groupby(['family', 'MotherID', 'FatherID']).count()
fam_not_in_phasing = []
parent_not_in_ihart = []
for fam, mother_id, father_id in family_idxs.index:
    if fam not in fams_phased:
        fam_not_in_phasing = fam_not_in_phasing + [fam]
        continue
    elif sum(bam_mappings.participant_id==mother_id)==0:
        parent_not_in_ihart = parent_not_in_ihart + [fam]
        continue
    elif sum(bam_mappings.participant_id==father_id)==0:
        parent_not_in_ihart = parent_not_in_ihart + [fam]
        continue    
    mother_sample = bam_mappings[bam_mappings.participant_id==mother_id].index[0]
    father_sample = bam_mappings[bam_mappings.participant_id==father_id].index[0]
    sib_samples = list(bam_mappings[(bam_mappings.FatherID==father_id) & (bam_mappings.MotherID==mother_id)].index)
    family_idx = '%s.%s.%s' % (fam, mother_id, father_id)
    fam_info_list = fam_info_list + [[family_idx, fam, mother_sample, father_sample, sib_samples]]

In [7]:
fam_info_df = pd.DataFrame(fam_info_list, columns = ['fam_identifier', 'fam', 'mother_sample', 'father_sample', 'sib_samples'])
fam_info_df.index=fam_info_df.fam
fam_info_df.to_pickle('/home/groups/dpwall/briannac/alt_haplotypes/data/phasings/fam_list.df')

In [24]:
pd.DataFrame(list(fam_info_df.mother_sample) + list(fam_info_df.father_sample) + list(np.concatenate(fam_info_df.sib_samples))).to_csv('/home/groups/dpwall/briannac/alt_haplotypes/data/phasings/samples.tsv', header=None, index=None)